# MCP Server Client - Jupyter Notebook

Interactive notebook for testing deployed MCP servers on GCP Cloud Run.

**Features:**
- Call 9 deployed MCP servers via Claude API
- Run bioinformatics workflows interactively
- Visualize results (spatial plots, pathway networks, heatmaps)
- Track token usage and costs
- Reproducible analysis workflows

**Prerequisites:**
- Python 3.11+
- ANTHROPIC_API_KEY environment variable set
- Internet connection (calls GCP Cloud Run servers)

---

## 1. Setup and Configuration

Import libraries and configure MCP server connections.

In [ ]:
# Install required packages (run once)
# !pip install anthropic python-dotenv pandas matplotlib seaborn plotly

In [ ]:
import os
import json
from typing import List, Dict, Any
import anthropic
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load environment variables
load_dotenv()

# Verify API key
api_key = os.getenv("ANTHROPIC_API_KEY")
if not api_key:
    raise ValueError("ANTHROPIC_API_KEY not found. Set it in .env file or environment.")

print("✅ API Key configured")
print(f"✅ Using Anthropic SDK version: {anthropic.__version__}")

## 2. MCP Server Configuration

All 9 MCP servers deployed on GCP Cloud Run.

In [ ]:
# MCP Server URLs (all deployed on GCP Cloud Run)
MCP_SERVERS = {
    "fgbio": {
        "url": "https://mcp-fgbio-ondu7mwjpa-uc.a.run.app/sse",
        "description": "Genomic reference data and FASTQ validation",
        "status": "production"
    },
    "multiomics": {
        "url": "https://mcp-multiomics-ondu7mwjpa-uc.a.run.app/sse",
        "description": "Multi-omics integration (RNA/Protein/Phospho)",
        "status": "production"
    },
    "spatialtools": {
        "url": "https://mcp-spatialtools-ondu7mwjpa-uc.a.run.app/sse",
        "description": "Spatial transcriptomics analysis",
        "status": "production"
    },
    "tcga": {
        "url": "https://mcp-tcga-ondu7mwjpa-uc.a.run.app/sse",
        "description": "TCGA cancer genomics data",
        "status": "mock"
    },
    "openimagedata": {
        "url": "https://mcp-openimagedata-ondu7mwjpa-uc.a.run.app/sse",
        "description": "H&E/MxIF image loading, registration, feature extraction, and composite generation",
        "status": "production"
    },
    "deepcell": {
        "url": "https://mcp-deepcell-ondu7mwjpa-uc.a.run.app/sse",
        "description": "DeepCell-TF cell segmentation and marker quantification",
        "status": "production"
    },
    "cell-classify": {
        "url": "https://mcp-cell-classify-ondu7mwjpa-uc.a.run.app/sse",
        "description": "Cell phenotype classification and visualization",
        "status": "production"
    },
    "mockepic": {
        "url": "https://mcp-mockepic-ondu7mwjpa-uc.a.run.app/sse",
        "description": "Mock EHR/FHIR data",
        "status": "mock"
    }
}

# Display available servers
print("Available MCP Servers:\n")
for name, config in MCP_SERVERS.items():
    status_icon = "+" if config["status"] == "production" else "~"
    print(f"{status_icon} {name}: {config['description']}")

## 3. Helper Functions

Utilities for calling MCP servers and processing responses.

In [ ]:
class MCPClient:
    """Helper class for calling MCP servers via Claude API."""
    
    def __init__(self, api_key: str = None):
        self.api_key = api_key or os.getenv("ANTHROPIC_API_KEY")
        self.client = anthropic.Anthropic(api_key=self.api_key)
        self.conversation_history = []
        
    def call_servers(
        self,
        prompt: str,
        servers: List[str],
        model: str = "claude-sonnet-4-5",
        max_tokens: int = 4096,
        clear_history: bool = False
    ) -> Dict[str, Any]:
        """Call MCP servers with a prompt.
        
        Args:
            prompt: User query/instruction
            servers: List of server names to enable
            model: Claude model to use
            max_tokens: Maximum response tokens
            clear_history: Clear conversation history before call
            
        Returns:
            Dict with response text, usage info, and metadata
        """
        if clear_history:
            self.conversation_history = []
            
        # Add user message to history
        self.conversation_history.append({
            "role": "user",
            "content": prompt
        })
        
        # Build MCP server configs
        mcp_servers = [
            {
                "type": "url",
                "url": MCP_SERVERS[server]["url"],
                "name": server
            }
            for server in servers
            if server in MCP_SERVERS
        ]
        
        # Build tools config
        tools = [
            {"type": "mcp_toolset", "mcp_server_name": server}
            for server in servers
            if server in MCP_SERVERS
        ]
        
        # Call Claude API
        response = self.client.beta.messages.create(
            model=model,
            max_tokens=max_tokens,
            messages=self.conversation_history,
            mcp_servers=mcp_servers,
            tools=tools,
            betas=["mcp-client-2025-11-20"]
        )
        
        # Extract response text
        response_text = ""
        for block in response.content:
            if hasattr(block, 'text'):
                response_text += block.text
        
        # Add assistant response to history
        self.conversation_history.append({
            "role": "assistant",
            "content": response_text
        })
        
        # Extract usage info
        usage = {
            "input_tokens": response.usage.input_tokens,
            "output_tokens": response.usage.output_tokens,
            "total_tokens": response.usage.input_tokens + response.usage.output_tokens,
            "estimated_cost_usd": self._estimate_cost(
                response.usage.input_tokens,
                response.usage.output_tokens,
                model
            )
        }
        
        return {
            "response": response_text,
            "usage": usage,
            "model": model,
            "servers_used": servers
        }
    
    def _estimate_cost(self, input_tokens: int, output_tokens: int, model: str) -> float:
        """Estimate cost in USD based on token usage."""
        # Pricing as of Dec 2025 (approximate)
        pricing = {
            "claude-sonnet-4-5": {"input": 0.003 / 1000, "output": 0.015 / 1000},
            "claude-opus-4-5": {"input": 0.015 / 1000, "output": 0.075 / 1000},
            "claude-haiku-4": {"input": 0.001 / 1000, "output": 0.005 / 1000}
        }
        
        rates = pricing.get(model, pricing["claude-sonnet-4-5"])
        cost = (input_tokens * rates["input"]) + (output_tokens * rates["output"])
        return round(cost, 4)
    
    def clear_history(self):
        """Clear conversation history."""
        self.conversation_history = []

# Initialize client
mcp = MCPClient()
print("✅ MCP Client initialized")

## 4. Quick Test - List Available Tools

Simple test to verify MCP server connectivity.

In [ ]:
# Quick test: Ask about available tools
result = mcp.call_servers(
    prompt="What spatial transcriptomics analysis tools are available?",
    servers=["spatialtools"],
    clear_history=True
)

print("📊 Response:\n")
print(result["response"])
print("\n" + "="*80 + "\n")
print(f"💰 Cost: ${result['usage']['estimated_cost_usd']}")
print(f"📈 Tokens: {result['usage']['input_tokens']} in, {result['usage']['output_tokens']} out")

## 5. Example Workflow: Spatial Transcriptomics Analysis

Complete workflow for analyzing spatial transcriptomics data.

In [ ]:
# Workflow 1: Cell Type Deconvolution
result = mcp.call_servers(
    prompt="""
    Analyze the spatial transcriptomics data for Patient-001.
    Perform cell type deconvolution and summarize the key cell populations.
    """,
    servers=["spatialtools", "mockepic"],
    clear_history=True
)

print("📊 Cell Type Deconvolution Results:\n")
print(result["response"])
print("\n" + "="*80 + "\n")
print(f"💰 Cost: ${result['usage']['estimated_cost_usd']}")

## 6. Example Workflow: Pathway Enrichment

Gene set enrichment analysis on differentially expressed genes.

In [ ]:
# Workflow 2: Pathway Enrichment
result = mcp.call_servers(
    prompt="""
    For the following upregulated genes: TP53, BRCA1, MYC, KRAS, PTEN
    Perform pathway enrichment analysis using GO_BP database.
    Show the top 5 enriched pathways.
    """,
    servers=["spatialtools"],
    clear_history=True
)

print("🧬 Pathway Enrichment Results:\n")
print(result["response"])
print("\n" + "="*80 + "\n")
print(f"💰 Cost: ${result['usage']['estimated_cost_usd']}")

## 7. Example Workflow: Multi-Omics Integration

Integrate RNA, protein, and phosphorylation data.

In [ ]:
# Workflow 3: Multi-omics Integration
result = mcp.call_servers(
    prompt="""
    Explain the multi-omics analysis workflow available.
    What tools can I use for integrating RNA, protein, and phosphorylation data?
    """,
    servers=["multiomics"],
    clear_history=True
)

print("🔬 Multi-Omics Integration Capabilities:\n")
print(result["response"])
print("\n" + "="*80 + "\n")
print(f"💰 Cost: ${result['usage']['estimated_cost_usd']}")

## 8. Complete PatientOne Workflow

End-to-end precision medicine analysis.

In [ ]:
# Complete workflow
result = mcp.call_servers(
    prompt="""
    For Patient-001 (ovarian cancer, platinum-resistant, on bevacizumab):
    
    1. Get clinical data
    2. Retrieve spatial transcriptomics data
    3. Perform cell type deconvolution
    4. Identify key findings for treatment planning
    
    Provide a summary of actionable insights.
    """,
    servers=["mockepic", "spatialtools"],
    clear_history=True,
    max_tokens=8192
)

print("🏥 PatientOne Complete Analysis:\n")
print(result["response"])
print("\n" + "="*80 + "\n")
print(f"💰 Cost: ${result['usage']['estimated_cost_usd']}")
print(f"📈 Tokens: {result['usage']['total_tokens']} total")

## 9. Token Usage Tracking

Monitor API costs across multiple queries.

In [ ]:
# Example: Track costs across multiple queries
queries = [
    ("What tools are available for spatial analysis?", ["spatialtools"]),
    ("Explain multi-omics integration methods.", ["multiomics"]),
    ("What genomic reference data can I access?", ["fgbio"])
]

total_cost = 0
results = []

for prompt, servers in queries:
    result = mcp.call_servers(prompt, servers, clear_history=True)
    total_cost += result["usage"]["estimated_cost_usd"]
    results.append({
        "query": prompt[:50] + "...",
        "servers": ", ".join(servers),
        "tokens": result["usage"]["total_tokens"],
        "cost_usd": result["usage"]["estimated_cost_usd"]
    })

# Display as DataFrame
df = pd.DataFrame(results)
print("💰 Token Usage Summary:\n")
print(df.to_string(index=False))
print(f"\n📊 Total Cost: ${total_cost:.4f}")

## 10. Visualization Example (Optional)

Visualize token usage across different queries.

In [ ]:
# Visualize token usage
if len(results) > 0:
    df = pd.DataFrame(results)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Token usage by query
    ax1.barh(df['query'], df['tokens'], color='skyblue')
    ax1.set_xlabel('Total Tokens')
    ax1.set_title('Token Usage by Query')
    ax1.grid(axis='x', alpha=0.3)
    
    # Cost by query
    ax2.barh(df['query'], df['cost_usd'], color='lightcoral')
    ax2.set_xlabel('Cost (USD)')
    ax2.set_title('API Cost by Query')
    ax2.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Average cost per query: ${df['cost_usd'].mean():.4f}")
    print(f"📊 Average tokens per query: {df['tokens'].mean():.0f}")

## 11. Advanced: Custom Workflow

Create your own analysis workflow by combining multiple MCP servers.

In [ ]:
# Custom workflow - modify this cell for your analysis
your_prompt = """
Your custom analysis prompt here...
"""

your_servers = ["spatialtools", "multiomics"]  # Select relevant servers

# Run your analysis
result = mcp.call_servers(
    prompt=your_prompt,
    servers=your_servers,
    clear_history=True
)

print(result["response"])
print(f"\n💰 Cost: ${result['usage']['estimated_cost_usd']}")

## Summary

This notebook provides:
- ✅ Direct access to 9 deployed MCP servers
- ✅ Helper functions for calling servers and tracking costs
- ✅ Example workflows for common bioinformatics tasks
- ✅ Visualization of token usage and costs
- ✅ Reproducible analysis environment

**Next Steps:**
- Modify workflows for your specific research questions
- Combine multiple MCP servers for complex analyses
- Export results to CSV/JSON for downstream processing
- Deploy to GCP Vertex AI Workbench for team collaboration

**Documentation:**
- [Main README](../../README.md)
- [MCP Server Docs](../../servers/)
- [Anthropic API Docs](https://docs.anthropic.com/)

---